In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasRegressor
seed = 42
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras import backend as K
K.set_image_dim_ordering('th')

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)
/home/cuda/.local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:

# import importlib
# import os
4077, -24.20391846],
       [ -3.06291199, -29.1925354 , -54.17045593, ..., -11.0681076 ,
        -25.03192139, -48.92904663],
       ..., 
       [ 12.9357605 ,   0.0642662 , -43.96864319, ...,  -5.27310944,
        -21.49401855, -50.50608826],
       [ -1.900383  , -20.00675201, -50.1171875 , ...,  -7.74325943,
        -23.73028564, -61.93417358],
       [ 16.03391647, -19.57675934, -28.65151978, ...,  -3.3750267 ,
        -10.91635132, -41.78013611]])
# def set_keras_backend(backend):

#     if K.backend() != backend:
#         os.environ['KERAS_BACKEND'] = backend
#         importlib.reload(K)
#         assert K.backend() == backend

# set_keras_backend("tensorflow")

Using TensorFlow backend.


In [2]:
data = pd.read_csv('keys.txt', delimiter="(",  header=None)
for i in range(5):
    data[i] = data[i].astype(str).str[:-1]
data.columns = ["image","Head","left_hand","steering_wheel","right_hand"]
data["Head_x"], data["Head_y"] = data['Head'].str.split(',', 1).str
data["left_hand_x"], data["left_hand_y"] = data['left_hand'].str.split(',', 1).str
data["steering_wheel_x"], data["steering_wheel_y"] = data['steering_wheel'].str.split(',', 1).str
data["right_hand_x"], data["right_hand_y"] = data['right_hand'].str.split(',', 1).str
data.drop("Head",axis=1,inplace=True)
data.drop("left_hand",axis=1,inplace=True)
data.drop("steering_wheel",axis=1,inplace=True)
data.drop("right_hand",axis=1,inplace=True)

In [7]:
data.head()

,image,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,img_56.jpg,192,178,461,126,443,219,587,321
1,img_63.jpg,333,135,636,56,581,171,637,383
2,img_118.jpg,242,134,539,161,540,240,633,398
3,img_120.jpg,163,132,479,110,440,213,592,304
4,img_138.jpg,192,185,514,105,490,213,588,407


In [4]:
data.to_csv("preprocessed.csv",sep="\t")

In [3]:
data["Head_x"] = (data["Head_x"].astype(int)*150/640).astype(int)

In [4]:
data["left_hand_x"] = (data["left_hand_x"].astype(int)*150/640).astype(int)
data["steering_wheel_x"] = (data["steering_wheel_x"].astype(int)*150/640).astype(int)
data["right_hand_x"] = (data["right_hand_x"].astype(int)*150/640).astype(int)

In [5]:
data["Head_y"] = (data["Head_y"].astype(int)*150/480).astype(int)
data["left_hand_y"] = (data["left_hand_y"].astype(int)*150/480).astype(int)
data["steering_wheel_y"] = (data["steering_wheel_y"].astype(int)*150/480).astype(int)
data["right_hand_y"] = (data["right_hand_y"].astype(int)*150/480).astype(int)

In [6]:
X = data["image"].values
data.drop("image",axis=1, inplace=True)
Y = data.values

In [7]:
x = np.array([np.array(Image.open(fname).resize((150,150),Image.ANTIALIAS)) for fname in "key_data/"+X])

In [8]:
x = x.transpose(0,3,1,2)
x.shape

(499, 3, 150, 150)

In [9]:
def key_model():
    model=Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3,150,150))),1
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    # the model so far outputs 3D feature maps (height, width, features)
    model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(32)) # 64 neurons
    model.add(Activation('relu'))

    # output layer: get 8 coordinate values
    model.add(Dense(8))
    model.compile(optimizer='rmsprop',
              loss='mse')
    return model

In [14]:
estimator = KerasRegressor(build_fn=key_model, nb_epoch=100, batch_size=100, verbose=True)

In [15]:
kfold = KFold(n_splits=10, random_state=seed)

In [16]:
results = cross_val_score(estimator, x, Y, cv=kfold)

/home/cuda/.local/lib/python3.5/site-packages/keras/backend/theano_backend.py:1890: UserWarning: dict_keys(['filter_dilation']) are now deprecated in `tensor.nnet.abstract_conv.conv2d` interface and will be ignored.
  filter_dilation=dilation_rate)


Epoch 1/10
449/449 [==============================] - 2s - loss: 161620253.8328     
Epoch 2/10
449/449 [==============================] - 2s - loss: 18746.5946     
Epoch 3/10
449/449 [==============================] - 2s - loss: 23061.7499     
Epoch 4/10
449/449 [==============================] - 2s - loss: 5797.5733     
Epoch 5/10
449/449 [==============================] - 2s - loss: 4860.5897     
Epoch 6/10
449/449 [==============================] - 2s - loss: 2987.6301     
Epoch 7/10
449/449 [==============================] - 2s - loss: 6299.9274     
Epoch 8/10
449/449 [==============================] - 2s - loss: 4440.2344     
Epoch 9/10
449/449 [==============================] - 2s - loss: 2480.8842     
Epoch 10/10
50/50 [==============================] - 0s
Epoch 1/10
449/449 [==============================] - 2s - loss: 77699053.9410     
Epoch 2/10
449/449 [==============================] - 2s - loss: 17410.8514     
Epoch 3/10
449/449 [==============================] 

In [17]:
results

array([  844.53131104,  3428.82983398,   615.43988037,   812.21459961,
        1587.42553711,  1887.31665039,  1323.55126953,  1084.23657227,
        2022.53198242,  2734.52099609])

In [18]:
Xprint("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Results: 1634.06 (859.33) MSE


In [19]:
estimator.fit(x, Y)
prediction = estimator.predict(x)

/home/cuda/.local/lib/python3.5/site-packages/keras/backend/theano_backend.py:1890: UserWarning: dict_keys(['filter_dilation']) are now deprecated in `tensor.nnet.abstract_conv.conv2d` interface and will be ignored.
  filter_dilation=dilation_rate)


Epoch 1/10
499/499 [==============================] - 2s - loss: 8136522.2136      
Epoch 2/10
499/499 [==============================] - 2s - loss: 7895.8009     
Epoch 3/10
499/499 [==============================] - 2s - loss: 15744.3975     
Epoch 4/10
499/499 [==============================] - 2s - loss: 4127.5210     
Epoch 5/10
499/499 [==============================] - 2s - loss: 1884.8105     
Epoch 6/10
499/499 [==============================] - 2s - loss: 1133.3692    
Epoch 7/10
499/499 [==============================] - 2s - loss: 887.0382     
Epoch 8/10
499/499 [==============================] - 2s - loss: 2895.0716     
Epoch 9/10
499/499 [==============================] - 2s - loss: 561.8583     
Epoch 10/10
499/499 [==============================] - 1s     


NameError: name 'y' is not defined

In [26]:
mse = Y-prediction
mse

array([[  5.44755173,  13.33124924, -35.94509888, ...,   0.36623383,
        -13.73826599, -49.1672821 ],
       [ 18.76260757,  -3.27388   ,  -7.21040344, ...,   2.66313553,
         -5.58554077, -24.20391846],
       [ -3.06291199, -29.1925354 , -54.17045593, ..., -11.0681076 ,
        -25.03192139, -48.92904663],
       ..., 
       [ 12.9357605 ,   0.0642662 , -43.96864319, ...,  -5.27310944,
        -21.49401855, -50.50608826],
       [ -1.900383  , -20.00675201, -50.1171875 , ...,  -7.74325943,
        -23.73028564, -61.93417358],
       [ 16.03391647, -19.57675934, -28.65151978, ...,  -3.3750267 ,
        -10.91635132, -41.78013611]])

In [27]:
mse = np.multiply(mse,mse)

array([  2.96758198e+01,   1.77722206e+02,   1.29205013e+03,
         5.27475861e+02,   1.36337359e+03,   1.34127215e-01,
         1.88739952e+02,   2.41742163e+03])